# Retinal Degeneration Associated With RPGRIP1


Data from [Beryozkin A, et al. Retinal Degeneration Associated With RPGRIP1: A Review of Natural History, Mutation Spectrum, and Genotype-Phenotype Correlation in 228 Patients](https://pubmed.ncbi.nlm.nih.gov/34722527)

In [1]:
import genophenocorr

print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


## Setup

### Load HPO

We use HPO `v2023-10-09` release for this analysis.

In [4]:
import hpotk

store = hpotk.configure_ontology_store()

hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')

Loaded HPO v2023-10-09


### Load Phenopackets

We will load phenopacket JSON files located in `phenopackets` folder that is next to the notebook.

In [5]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'phenopackets'
cohort_creator = configure_caching_cohort_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

Patients Created: 100%|██████████| 229/229 [00:00<00:00, 1831.85it/s]
Validated under none policy
Showing errors and warnings
229 phenopacket(s) found at `phenopackets`
  patient #0
    phenotype-features
     warnings:
     ·No diseases found.
  patient #1
    phenotype-features
     warnings:
     ·No diseases found.
  patient #2
    phenotype-features
     warnings:
     ·No diseases found.
  patient #3
    phenotype-features
     warnings:
     ·No diseases found.
  patient #4
    phenotype-features
     warnings:
     ·No diseases found.
  patient #5
    phenotype-features
     warnings:
     ·No diseases found.
  patient #6
    phenotype-features
     warnings:
     ·No diseases found.
  patient #7
    phenotype-features
     warnings:
     ·No diseases found.
  patient #8
    phenotype-features
     warnings:
     ·No diseases found.
  patient #9
    phenotype-features
     warnings:
     ·No diseases found.
  patient #10
    phenotype-features
     warnings:
     ·No diseases f

### Pick transcript

We choose the [MANE Select](https://www.ncbi.nlm.nih.gov/nuccore/NM_020366.4) transcript for *RPGRIP1*.

In [6]:
tx_id = 'NM_020366.4'

## Explore cohort

Explore the cohort to guide selection of the genotype-phenotype analysis.


In [9]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewable

viewer = CohortViewable(hpo)

In [12]:
display(HTML(viewer.process(cohort=cohort, transcript_id=tx_id)))

HPO Term,ID,Annotation Count
Very low visual acuity,HP:0032122,89
Eye poking,HP:0001483,28
Abnormal electroretinogram,HP:0000512,28
Moderately reduced visual acuity,HP:0030515,24
Global developmental delay,HP:0001263,12
Moderate hypermetropia,HP:0031729,12
Constriction of peripheral visual field,HP:0001133,10
High hypermetropia,HP:0008499,9
Severely reduced visual acuity,HP:0001141,6
High myopia,HP:0011003,4


In [8]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

Variant,Effect,Count,Key
c.1107del,FRAMESHIFT_VARIANT,25,14_21312457_21312458_GA_G
c.2480G>T,MISSENSE_VARIANT,12,14_21325943_21325943_G_T
c.535del,FRAMESHIFT_VARIANT,8,14_21302530_21302531_AG_A
c.3565C>T,STOP_GAINED,8,14_21345145_21345145_C_T
c.2236G>A,MISSENSE_VARIANT,7,14_21325252_21325252_G_A
c.3565_3571del,FRAMESHIFT_VARIANT,7,14_21345139_21345146_CAAGGCCG_C
c.2759_2760insT,FRAMESHIFT_VARIANT,6,14_21327671_21327671_A_AT
c.2249A>G,MISSENSE_VARIANT,5,14_21325265_21325265_A_G
c.1180C>T,STOP_GAINED,5,14_21317724_21317724_C_T
c.799C>T,STOP_GAINED,5,14_21303542_21303542_C_T


## Configure the analysis

In [13]:
from genophenocorr.analysis import configure_cohort_analysis

analysis = configure_cohort_analysis(cohort, hpo)

## Run the analyses

Test for presence of genotype-phenotype correlations between missense variants vs. others.

In [14]:
from genophenocorr.model import VariantEffect
from genophenocorr.analysis.predicate import PatientCategories
from genophenocorr.analysis.predicate.genotype import VariantEffectPredicate

by_missense = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=tx_id)
by_missense.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_020366.4                       Yes             No  \
                                                    Count Percent  Count   
Eye poking [HP:0001483]                               3/5     60%  25/34   
All [HP:0000001]                                    24/24    100%  97/97   
Visual field defect [HP:0001123]                      1/1    100%    9/9   
Abnormal visual electrophysiology [HP:0030453]        4/4    100%  23/23   
Very low visual acuity [HP:0032122]                 15/15    100%  66/66   
Abnormal eye physiology [HP:0012373]                24/24    100%  95/95   
Moderate hypermetropia [HP:0031729]                   3/3    100%    9/9   
Tactile sensory seeking [HP:5200065]                  3/3    100%  25/25   
Abnormal electroretinogram [HP:0000512]               4/4    100%  23/24   
High myopia [HP:0011003]                              0/0      0%    4/4   
Visual impairment [HP:0000505]                      24/24    100%  90/90   
Restricted or repetitive behaviors or interests...    3/3    100%  25/25   
Abnormality of refraction [HP:0000539]                4/4    100%  24/24   
Abnormality of the nervous system [HP:0000707]        3/3    100%  32/32   
Myopia [HP:0000545]                                   0/0      0%    4/4   
Abnormality of vision [HP:0000504]                  24/24    100%  90/90   
Neurodevelopmental delay [HP:0012758]                 0/0      0%  11/11   
Reduced visual acuity [HP:0007663]                  24/24    100%  90/90   
Sensory seeking [HP:4000079]                          3/3    100%  25/25   
Moderately reduced visual acuity [HP:0030515]         9/9    100%  15/15   
Abnormality of mental function [HP:0011446]           3/3    100%  25/25   
Abnormal repetitive mannerisms [HP:0000733]           3/3    100%  25/25   
Severely reduced visual acuity [HP:0001141]           0/0      0%    6/6   
Mildly reduced visual acuity [HP:0032037]             0/0      0%    3/3   
Hypermetropia [HP:0000540]                            4/4    100%  20/20   
Recurrent maladaptive behavior [HP:5200241]           3/3    100%  25/25   
Neurodevelopmental abnormality [HP:0012759]           0/0      0%  11/11   
Atypical behavior [HP:0000708]                        3/3    100%  25/25   
Mild hypermetropia [HP:0031728]                       1/1    100%    1/1   
Abnormality of the eye [HP:0000478]                 24/24    100%  95/95   
Self-injurious behavior [HP:0100716]                  3/3    100%  25/25   
Phenotypic abnormality [HP:0000118]                 24/24    100%  97/97   
Abnormal nervous system physiology [HP:0012638]       3/3    100%  32/32   
High hypermetropia [HP:0008499]                       0/0      0%    9/9   
Constriction of peripheral visual field [HP:000...    1/1    100%    9/9   
Abnormal movements of face and head [HP:5200017]      3/3    100%  25/25   
Sensory behavioral abnormality [HP:5200046]           3/3    100%  25/25   

MISSENSE_VARIANT on NM_020366.4                                       \
                                                   Percent   p value   
Eye poking [HP:0001483]                                74%  0.608819   
All [HP:0000001]                                      100%  1.000000   
Visual field defect [HP:0001123]                      100%  1.000000   
Abnormal visual electrophysiology [HP:0030453]        100%  1.000000   
Very low visual acuity [HP:0032122]                   100%  1.000000   
Abnormal eye physiology [HP:0012373]                  100%  1.000000   
Moderate hypermetropia [HP:0031729]                   100%  1.000000   
Tactile sensory seeking [HP:5200065]                  100%  1.000000   
Abnormal electroretinogram [HP:0000512]                96%  1.000000   
High myopia [HP:0011003]                              100%  1.000000   
Visual impairment [HP:0000505]                        100%  1.000000   
Restricted or repetitive behaviors or interests...    100%  1.000000   
Abnormality of refraction [

Test for presence of genotype-phenotype correlations between frameshift variants vs. others.

In [15]:
by_frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id=tx_id)
by_frameshift.summarize(hpo, PatientCategories.YES)

FRAMESHIFT_VARIANT on NM_020366.4                     Yes             No  \
                                                    Count Percent  Count   
Eye poking [HP:0001483]                             23/27     85%   5/12   
All [HP:0000001]                                    52/52    100%  69/69   
Visual field defect [HP:0001123]                      5/5    100%    5/5   
Abnormal visual electrophysiology [HP:0030453]      11/11    100%  16/16   
Very low visual acuity [HP:0032122]                 36/36    100%  45/45   
Abnormal eye physiology [HP:0012373]                52/52    100%  67/67   
Moderate hypermetropia [HP:0031729]                   7/7    100%    5/5   
Tactile sensory seeking [HP:5200065]                23/23    100%    5/5   
Abnormal electroretinogram [HP:0000512]             11/11    100%  16/17   
High myopia [HP:0011003]                              4/4    100%    0/0   
Visual impairment [HP:0000505]                      48/48    100%  66/66   
Restricted or repetitive behaviors or interests...  23/23    100%    5/5   
Abnormality of refraction [HP:0000539]              16/16    100%  12/12   
Abnormality of the nervous system [HP:0000707]      27/27    100%    8/8   
Myopia [HP:0000545]                                   4/4    100%    0/0   
Abnormality of vision [HP:0000504]                  48/48    100%  66/66   
Neurodevelopmental delay [HP:0012758]                 8/8    100%    3/3   
Reduced visual acuity [HP:0007663]                  48/48    100%  66/66   
Sensory seeking [HP:4000079]                        23/23    100%    5/5   
Moderately reduced visual acuity [HP:0030515]         8/8    100%  16/16   
Abnormality of mental function [HP:0011446]         23/23    100%    5/5   
Abnormal repetitive mannerisms [HP:0000733]         23/23    100%    5/5   
Severely reduced visual acuity [HP:0001141]           3/3    100%    3/3   
Mildly reduced visual acuity [HP:0032037]             1/1    100%    2/2   
Hypermetropia [HP:0000540]                          12/12    100%  12/12   
Recurrent maladaptive behavior [HP:5200241]         23/23    100%    5/5   
Neurodevelopmental abnormality [HP:0012759]           8/8    100%    3/3   
Atypical behavior [HP:0000708]                      23/23    100%    5/5   
Mild hypermetropia [HP:0031728]                       1/1    100%    1/1   
Abnormality of the eye [HP:0000478]                 52/52    100%  67/67   
Self-injurious behavior [HP:0100716]                23/23    100%    5/5   
Phenotypic abnormality [HP:0000118]                 52/52    100%  69/69   
Abnormal nervous system physiology [HP:0012638]     27/27    100%    8/8   
High hypermetropia [HP:0008499]                       4/4    100%    5/5   
Constriction of peripheral visual field [HP:000...    5/5    100%    5/5   
Abnormal movements of face and head [HP:5200017]    23/23    100%    5/5   
Sensory behavioral abnormality [HP:5200046]         23/23    100%    5/5   

FRAMESHIFT_VARIANT on NM_020366.4                                     \
                                                   Percent   p value   
Eye poking [HP:0001483]                                42%  0.016983   
All [HP:0000001]                                      100%  1.000000   
Visual field defect [HP:0001123]                      100%  1.000000   
Abnormal visual electrophysiology [HP:0030453]        100%  1.000000   
Very low visual acuity [HP:0032122]                   100%  1.000000   
Abnormal eye physiology [HP:0012373]                  100%  1.000000   
Moderate hypermetropia [HP:0031729]                   100%  1.000000   
Tactile sensory seeking [HP:5200065]                  100%  1.000000   
Abnormal electroretinogram [HP:0000512]                94%  1.000000   
High myopia [HP:0011003]                                0%  1.000000   
Visual impairment [HP:0000505]                        100%  1.000000   
Restricted or repetitive behaviors or interests...    100%  1.000000   
Abnormality of refraction [

Or between subjects with >=1 allele of a variant vs. the other subjects:

In [16]:
variant_key = '14_21312457_21312458_GA_G'

by_var = analysis.compare_by_variant_key(variant_key)
by_var.summarize(hpo, PatientCategories.YES)

>=1 allele of the variant 14_21312457_21312458_GA_G    Yes               No  \
                                                     Count Percent    Count   
Eye poking [HP:0001483]                              16/16    100%    12/23   
All [HP:0000001]                                     17/17    100%  104/104   
Visual field defect [HP:0001123]                       0/0      0%    10/10   
Abnormal visual electrophysiology [HP:0030453]         0/0      0%    27/27   
Very low visual acuity [HP:0032122]                  17/17    100%    64/64   
Abnormal eye physiology [HP:0012373]                 17/17    100%  102/102   
Moderate hypermetropia [HP:0031729]                    0/0      0%    12/12   
Tactile sensory seeking [HP:5200065]                 16/16    100%    12/12   
Abnormal electroretinogram [HP:0000512]                0/0      0%    27/28   
High myopia [HP:0011003]                               0/0      0%      4/4   
Visual impairment [HP:0000505]                       17/17    100%    97/97   
Restricted or repetitive behaviors or interests...   16/16    100%    12/12   
Abnormality of refraction [HP:0000539]                 3/3    100%    25/25   
Abnormality of the nervous system [HP:0000707]       16/16    100%    19/19   
Myopia [HP:0000545]                                    0/0      0%      4/4   
Abnormality of vision [HP:0000504]                   17/17    100%    97/97   
Neurodevelopmental delay [HP:0012758]                  4/4    100%      7/7   
Reduced visual acuity [HP:0007663]                   17/17    100%    97/97   
Sensory seeking [HP:4000079]                         16/16    100%    12/12   
Moderately reduced visual acuity [HP:0030515]          0/0      0%    24/24   
Abnormality of mental function [HP:0011446]          16/16    100%    12/12   
Abnormal repetitive mannerisms [HP:0000733]          16/16    100%    12/12   
Severely reduced visual acuity [HP:0001141]            0/0      0%      6/6   
Mildly reduced visual acuity [HP:0032037]              0/0      0%      3/3   
Hypermetropia [HP:0000540]                             3/3    100%    21/21   
Recurrent maladaptive behavior [HP:5200241]          16/16    100%    12/12   
Neurodevelopmental abnormality [HP:0012759]            4/4    100%      7/7   
Atypical behavior [HP:0000708]                       16/16    100%    12/12   
Mild hypermetropia [HP:0031728]                        0/0      0%      2/2   
Abnormality of the eye [HP:0000478]                  17/17    100%  102/102   
Self-injurious behavior [HP:0100716]                 16/16    100%    12/12   
Phenotypic abnormality [HP:0000118]                  17/17    100%  104/104   
Abnormal nervous system physiology [HP:0012638]      16/16    100%    19/19   
High hypermetropia [HP:0008499]                        3/3    100%      6/6   
Constriction of peripheral visual field [HP:000...     0/0      0%    10/10   
Abnormal movements of face and head [HP:5200017]     16/16    100%    12/12   
Sensory behavioral abnormality [HP:5200046]          16/16    100%    12/12   

>=1 allele of the variant 14_21312457_21312458_GA_G                    \
                                                    Percent   p value   
Eye poking [HP:0001483]                                 52%  0.000919   
All [HP:0000001]                                       100%  1.000000   
Visual field defect [HP:0001123]                       100%  1.000000   
Abnormal visual electrophysiology [HP:0030453]         100%  1.000000   
Very low visual acuity [HP:0032122]                    100%  1.000000   
Abnormal eye physiology [HP:0012373]                   100%  1.000000   
Moderate hypermetropia [HP:0031729]                    100%  1.000000   
Tactile sensory seeking [HP:5200065]                   100%  1.000000   
Abnormal electroretinogram [HP:0000512]                 96%  1.000000   
High myopia [HP:0011003]                               100%  1.000000   
Visual impairment [HP:0000505]            

In [17]:
by_dis = analysis.compare_disease_vs_genotype("OMIM:608194",
                                              VariantEffectPredicate(transcript_id=tx_id, effect=VariantEffect.FRAMESHIFT_VARIANT))
by_dis.summarize(hpo, PatientCategories.YES)

TypeError: object of type 'VariantEffectPredicate' has no len()

In [14]:
cohort.list_all_diseases()

[('OMIM:613826', 183), ('OMIM:608194', 37)]

TODO - finalize!